In [13]:
import pandas as pd
import os
import chess.pgn

from   stockfish import Stockfish
from   io        import StringIO
from   pathlib   import Path
from   shutil    import copyfile
from   concurrent.futures import ProcessPoolExecutor

# Path to Stockfish binary (adjust as needed)
stockfish_path = "Engines/Stockfish"

# Depth for Stockfish evaluation (adjust as needed, e.g., 20 for 20-ply depth)
depth = 20

# Initialize Stockfish engine
stockfish = Stockfish(path=stockfish_path, depth=depth)

# Directory containing original partitions
storage_directory = "/Users/Macington/Documents/Projects/Project Scotch/Games/Storage"

# Directory to write revised partitions
storage_test_directory = "Games/Storage_test"

# Create the test directory if it doesn't exist
os.makedirs(storage_test_directory, exist_ok=True)

# Iterate through each total_ply directory
for total_ply in range(22, 350):
    print(f"Processing total_ply={total_ply}...")

    # Define paths
    partition_path = Path(storage_directory, f"total_ply={total_ply}", "data.parquet")
    num_rows_path = Path(storage_directory, f"total_ply={total_ply}", "num_rows.txt")

    # Check if the partition file exists
    if not partition_path.exists():
        print(f"Partition file {partition_path} does not exist. Skipping...")
        continue

    # Read the partition
    df = pd.read_parquet(partition_path)

    # Create a list to store evaluations
    evaluations = []

    # Iterate through all unique games
    for game_id in df['game_id'].unique():
        game_pgn = df[df['game_id'] == game_id]['pgn'].iloc[0]
        pgn_io = StringIO(game_pgn)  # Wrap the PGN string in a StringIO object
        game = chess.pgn.read_game(pgn_io)  # Pass the StringIO object to read_game

        # Set up a chess board
        board = game.board()

        # Iterate through all plies (half-moves) in the game
        for move in game.mainline_moves():
            board.push(move)
            stockfish.set_fen_position(board.fen())
            centipawn_evaluation = stockfish.get_evaluation()['value']
            evaluations.append((game_id, board.ply(), centipawn_evaluation))

    # Convert evaluations to DataFrame
    eval_df = pd.DataFrame(evaluations, columns=['game_id', 'ply', 'centipawn_evaluation'])

    # Merge evaluations with original DataFrame
    df = df.merge(eval_df, on=['game_id', 'ply'], how='left')

    # Create new directory if needed
    new_partition_directory = os.path.dirname(partition_path.as_posix().replace(storage_directory, storage_test_directory))
    os.makedirs(new_partition_directory, exist_ok=True)

    # Write the revised partition
    new_partition_path = os.path.join(new_partition_directory, "data.parquet")
    df.to_parquet(new_partition_path)

    # Copy num_rows.txt file to the new directory
    if num_rows_path.exists():
        new_num_rows_path = os.path.join(new_partition_directory, "num_rows.txt")
        copyfile(num_rows_path, new_num_rows_path)
        print(f"Copied {num_rows_path} to {new_num_rows_path}")
    else:
        print(f"File {num_rows_path} does not exist. Skipping...")

print("Processing completed!")


Processing total_ply=1...
Copied Games/Storage/total_ply=1/num_rows.txt to Games/Storage_test/total_ply=1/num_rows.txt
Processing total_ply=2...
Copied Games/Storage/total_ply=2/num_rows.txt to Games/Storage_test/total_ply=2/num_rows.txt
Processing total_ply=3...
Copied Games/Storage/total_ply=3/num_rows.txt to Games/Storage_test/total_ply=3/num_rows.txt
Processing total_ply=4...
Partition file Games/Storage/total_ply=4/data.parquet does not exist. Skipping...
Processing total_ply=5...
Copied Games/Storage/total_ply=5/num_rows.txt to Games/Storage_test/total_ply=5/num_rows.txt
Processing total_ply=6...
Copied Games/Storage/total_ply=6/num_rows.txt to Games/Storage_test/total_ply=6/num_rows.txt
Processing total_ply=7...
Copied Games/Storage/total_ply=7/num_rows.txt to Games/Storage_test/total_ply=7/num_rows.txt
Processing total_ply=8...
Copied Games/Storage/total_ply=8/num_rows.txt to Games/Storage_test/total_ply=8/num_rows.txt
Processing total_ply=9...
Copied Games/Storage/total_ply=9/

KeyboardInterrupt: 